# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [56]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [63]:
# Incorporated citipy to determine city based on latitude and longitude
# Study data files
weather_metadata = "my_weather_data.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(weather_metadata)

# Combine the data into a single dataset
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()

,Unnamed: 0,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Maximum Temp,Wind Speed
0,0,Rikitea,PF,100,1570568400,83,-23.1203,-134.9692,67.14,10.45
1,1,Kaitangata,NZ,100,1570568400,97,-46.2818,169.8464,49.96,2.79
2,2,Omboue,GA,90,1570568400,95,-1.5747,9.2618,75.02,2.72
3,3,Butaritari,KI,33,1570568400,66,3.0707,172.7902,85.34,12.80
4,4,Arraial do Cabo,BR,100,1570568400,93,-22.9663,-42.0245,75.06,3.40


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [64]:
gmaps.configure(api_key=g_key)

In [66]:
# Store 'Lat' and 'Lng' into  locations 
locations = weatherdf[["Latitude", "Longitude"]]

humidityrate = weatherdf["Humidity"]

In [67]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [40]:
#Narrow down the DataFrame to find your ideal weather condition.

#A max temperature lower than 80 degrees but higher than 70.
first = weatherdf.loc[weatherdf["Maximum Temp"]<80,:]
second = first.loc[weatherdf["Maximum Temp"]>70,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed"]<10,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()

,Unnamed: 0,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Maximum Temp,Wind Speed
23,23,Lagoa,PT,0,1570568400,77,37.1363,-8.4541,70.70,7.27
35,35,Nixa,US,0,1570568400,40,37.0434,-93.2944,71.67,7.63
45,45,Shakawe,BW,0,1570568400,14,-18.3633,21.8398,73.98,6.67
50,50,Pilar,AR,0,1570568400,59,-34.4571,-58.9142,72.66,5.17
75,75,Maldonado,UY,0,1570568400,63,-34.9088,-54.9583,70.39,7.40


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Latitude']
hotel_df['Lng'] = fifth['Longitude']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng
23,Lagoa,PT,37.1363,-8.4541
35,Nixa,US,37.0434,-93.2944
45,Shakawe,BW,-18.3633,21.8398
50,Pilar,AR,-34.4571,-58.9142
75,Maldonado,UY,-34.9088,-54.9583


In [42]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 23: Lagoa.
Closest hotel to Lagoa at 37.1363 , -8.4541 is QUINTA DO PARAISO.
------------
Retrieving Results for Index 35: Nixa.
Closest hotel to Nixa at 37.0434 , -93.2944 is Dollar General.
------------
Retrieving Results for Index 45: Shakawe.
Closest hotel to Shakawe at -18.3633 , 21.8398 is Archein Hardware, Shakawe.
------------
Retrieving Results for Index 50: Pilar.
Closest hotel to Pilar at -34.4571 , -58.9142 is Suites del Pilar.
------------
Retrieving Results for Index 75: Maldonado.
Closest hotel to Maldonado at -34.9088 , -54.9583 is De La Plaza Hotel.
------------
Retrieving Results for Index 93: Marzuq.
Closest hotel to Marzuq at 14.4 , 46.4667 is تموينات وادي عبدان.
------------
Retrieving Results for Index 100: Nacala.
Closest hotel to Nacala at -14.5565 , 40.687 is Norteventos.
------------
Retrieving Results for Index 120: Olga.
Closest hotel to Olga at 34.1125 , -118.1657 is The Westin Pasadena.
------------
Retrieving Results for Index

In [43]:
hotel_df.head()


,City,Country,Lat,Lng,Hotel Name
23,Lagoa,PT,37.1363,-8.4541,QUINTA DO PARAISO
35,Nixa,US,37.0434,-93.2944,Dollar General
45,Shakawe,BW,-18.3633,21.8398,"Archein Hardware, Shakawe"
50,Pilar,AR,-34.4571,-58.9142,Suites del Pilar
75,Maldonado,UY,-34.9088,-54.9583,De La Plaza Hotel


In [70]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
"""info_box_template = 
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [71]:
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=humidityrate, 
                                 dissipating=False, max_intensity=50,
                                 point_radius = 2)

marker_layer = gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("vacationheatmapwithhotels.png")

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>